```
Summary by:
- A41316 - Nguyễn Hữu Khoa
- A42718 - Lê Thảo Quyên
```

> **Note:** Many source code is outdated, some parameters, function usage have been changed according to elasticsearch and kibana 8.8.1

# Data Collection

### Library dependencies

In [1]:
import praw
import sqlite3

### Sqlite storage setup

In [2]:
conn = sqlite3.connect('reddit.db')
c = conn.cursor()

In [3]:
c.execute('''DROP TABLE IF EXISTS topics''')

c.execute('''DROP TABLE IF EXISTS comments''')

c.execute('''CREATE TABLE topics
             (topicTitle text, topicText text, topicID text, topicCategory text)''')

c.execute('''CREATE TABLE comments
             (commentText text, commentID text ,topicTitle text, topicText text, topicID text , topicCategory text)''')

### Praw Reddit connection information

In [4]:
user_agent = "Introducing Data Science Book"
r = praw.Reddit(user_agent=user_agent)
subreddits = ['datascience','gameofthrones']
limit = 1000

r.set_oauth_app_info(client_id='stJlUSUbPQe5lQ',
...                      client_secret='DoNotSHAREWithANYBODY',
...                      redirect_uri='http://127.0.0.1:65010/'
...                                   'authorize_callback')

MissingRequiredAttributeException: Required configuration setting 'client_id' missing. 
This setting can be provided in a praw.ini file, as a keyword argument to the Reddit class constructor, or as an environment variable.